# 3D data prep
ArcGIS Pro Python script to process LAS and NAIP layers in preparation for 3D analysis and visualization. This script requires that you use this geodatabase containing index layers for downloading assets.

In [ ]:
# Import ArcGIS package
import arcpy
# Subprocess allows us to issue commands on the command line
import subprocess
# Module to download files with an URL
from urllib.request import urlretrieve
# Zip utlity to extract files
from zipfile import ZipFile
# pandas to view table and possibly analyze data in the future?
import pandas as pd

## Instructions
* Pick a location anywhere in Kentucky and drop a point in ArcGIS.
* Download and extract [download_grids.gdb.zip](../pointcloud_extract/download_grids.gdb.zip)

## Cumberland Falls example
![cufa](../pointcloud_extract/cufa.jpg)


In [ ]:
# Directory for project 
# Got to use double backslash because of other modules
out_directory = "Z:\\BoydsGIS\\L7\\"

# Project name - creates a folder in project directoru
project = "cfalls"

# Name of point layer and buffer distance from point
point_name = "cumberland_falls"
buffer_distance = 1000

# Output geodatabase name
out_geodb = "workspace.gdb"

# Locations of index grids
las_grid = "Z:/BoydsGIS/data/download_grids.gdb/KY_5k_PointClound_grid"
naip_grid = "Z:/BoydsGIS/data/download_grids.gdb/Kentucky_10K_NAIP"

# NAIP files have prefix
naip_prefix = "ky_2ft_naip_2016_"

# Point the script to the directory with the laszip64.exe
las_tools = ["Z:\\BoydsGIS\\data\\lidar\\", "laszip64.exe"]

# Downloads folder
downloads = f'{out_directory}{project}\\downloads\\'
lidar = f'{out_directory}{project}\\lidar\\'
lidar_color = f'{out_directory}{project}\\lidar_color\\'

# LAS dataset name, temp list, and class codes
las_dataset = f'{lidar}{project}.lasd'
las_names = []
las_ground = [2, 9]


![cufa](../pointcloud_extract/cufa.gif)

In [ ]:
# Create folders and copy laszip64.exe
subprocess.run(f'mkdir {out_directory}{project}', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\downloads', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar_color', shell=True)
completed = subprocess.run(f'dir', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))
completed = subprocess.run(f'copy {las_tools[0]}{las_tools[1]} {downloads}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
# Create project geodatabase
arcpy.CreateFileGDB_management(f'{out_directory}{project}', out_geodb)

In [ ]:
# Create project default geodatabase
out_path = f'{out_directory}{project}\\{out_geodb}'
arcpy.env.workspace = out_path

In [ ]:
# Create empty point layer
spatial_reference = arcpy.Describe(source_grid).spatialReference
arcpy.CreateFeatureclass_management(out_path, point_name, "POINT", "#", "#", "#", spatial_reference)

## Open ArcGIS Pro

What location do you want? Edit the empty point layer by dropping a **single** point somewhere in Kentucky. Save the edit.

In [ ]:
# Buffer point
arcpy.Buffer_analysis(point_name, f'{point_name}_{buffer_distance}ft', buffer_distance)

In [ ]:
# Create a temp layer to find which LAS files to download
arcpy.Intersect_analysis ([f'{point_name}_{buffer_distance}ft', las_grid], "temp")

In [ ]:
# Find URLs and download them and use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
i = 0
las_names = []
for row in cursor:
    url = row[0]
    name = url[-12:]
    las_names.append(f'{lidar}{url[-12:-4]}.las')
    print(las_names)
    urlretrieve(url, f'{downloads}{name}')
    completed = subprocess.run(f'{downloads}{las_tools[1]} -i {downloads}{name} -o {lidar}{las_names[i]}', shell=True, stdout=subprocess.PIPE)
    print(completed.stdout.decode('UTF-8'))
    i += 1

In [ ]:
arcpy.CreateLasDataset_management (las_names, las_dataset, "#", "#", spatial_reference, True, True)

In [ ]:
arcpy.LasDatasetStatistics_management (las_dataset, "#", f'{out_directory}{project}\\stats.csv', "#", "#", "#")
with open(f'{out_directory}{project}\\stats.csv', encoding='utf-8') as csv:
    reader = pd.read_csv(csv)
    # Create pandas data frame that
    pdData = pd.DataFrame(reader)

pdData[pdData["Category"] == "ClassCodes"]

# Decide which class codes to add/change
Use the table above and decide which classes to include. Some parts of the state have more detailed classes. E.g., Louisville has building codes.

In [ ]:
# default ground classes
las_ground = [2, 9]

# Filter for ground points and create DEM and hillshade
arcpy.MakeLasDatasetLayer_management (las_dataset, f'{lidar}ground', las_ground)
arcpy.LasDatasetToRaster_conversion (f'{lidar}ground', f'{project}_dem_5ft', "#", "#", "#", "#", 5)
arcpy.HillShade_3d(f'{project}_dem_5ft', f'{project}_hillshade', 270, 55)

In [ ]:
# Create a temp layer to find which NAIP files to download
arcpy.RasterDomain_3d (f'{project}_hillshade', 'domain', 'POLYGON')
arcpy.Intersect_analysis (['domain', naip_grid], "temp")

In [ ]:
# Find URLs, download them and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
i = 0
naip_names = []
for row in cursor:
    url = row[0]
    name = row[1]
    naip_names.append(name)
    print(naip_names)
    urlretrieve(url, f'{downloads}{name}.zip')
    with ZipFile(f'{downloads}{name}.zip', 'r') as zip: 
        zip.extractall(f'{downloads}{name}') 
    arcpy.CopyRaster_management (f'{downloads}{name}\\{naip_prefix}{name}.jpg', name)
    i += 1

In [ ]:
# If multiple NAIPs, then mosaic to new raster and clip
if len(naip_names) > 1:
    arcpy.MosaicToNewRaster_management (naip_names, out_path, "temp", "#", "#", "#", 3. "#", "#")
    arcpy.Clip_management ('temp', '#', f'{project}_naip', 'domain')
else:
    arcpy.Clip_management (naip_names[0], '#', f'{project}_naip', 'domain')


In [ ]:
# Extract LAS points in buffer and colorize
arcpy.ExtractLas_3d (las_dataset, lidar, f'{point_name}_{buffer_distance}ft', "#", "#", "_extract", "#", "#", True, f'{lidar_local}temp.lasd')
arcpy.ColorizeLas_3d (f'{lidar_local}temp.lasd', f'{project}_naip', 'RED Band_1; GREEN Band_2; BLUE Band_3', lidar_color, "_color", "#",  "#",  "#",  "#", True, f'{lidar_color}{project}_rgb.lasd')

In [ ]:
# Create DSM of cliffs over 30 feet in 30-ft diameter neighborhood from bare-earth DEM
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "RANGE")
outFocalStat.save("focal_stats_30ft")
cliffs_over_30ft = arcpy.sa.Con(outFocalStat > 30, outFocalStat)
cliffs_over_30ft.save("cliffs_over_30ft")